In [10]:
import question_template
import re

In [11]:
templateGen = question_template.genQuesOneKPI('month')

In [12]:
templateGen

'cho tui biết t<tháng>/<năm> chỉ tiêu <tên chỉ tiêu> này được <tổng công ty> điều hành như thế nào rồi ?'

In [13]:
def find(s, ch):
    return [i for i, ltr in enumerate(s) if ltr == ch]

In [14]:
lowerDigit = find(templateGen,'<')
greaterDigit = find(templateGen,'>')

In [23]:
print(lowerDigit)
print(greaterDigit)
lenDigit = [g + 1 - l for l,g in zip(lowerDigit,greaterDigit)]
print(lenDigit)

[1, 9, 58, 78]
[7, 13, 71, 91]
[7, 5, 14, 14]


In [26]:
month = "1"
year = "2022"
kpi = "tỉ lệ phản ánh CSKH"
comp = "VTS"
quarter = ""
nextMonth = "2"
season = "3"
unit = "%"
groupKPI = ""

In [2]:
meanDesc = ['mean','trung bình','bình quân','giá trị trung bình','trung bình cộng']
minDesc = ['giá trị nhỏ nhất','min','giá trị bé nhất','giá trị tối thiểu','giá trị thấp nhất']
maxDesc = ['giá trị lớn nhất','max','giá trị tối đa','giá trị bự nhất','giá trị to nhất']
datList = ['đạt','ổn','tốt','hoàn thành','như mong đợi']
koDatList = ['chưa đạt','không đạt','không ổn','chưa tốt','không hoàn thành','chưa hoàn thành','chưa ổn','không ổn','không như mong đợi','chưa như mong đợi']
allList = ['tất cả','toàn bộ','các','đầy đủ','bao gồm']
detailList = ["chi tiết","sâu hơn","tường tận","cặn kẽ","tỉ mỉ","đầy đủ","tường minh","rõ ràng"]
overallList = ["khái quát","tổng quan","tổng thể","cơ bản","tổng hợp","tóm tắt"]
nextMonthList = ["tháng tiếp theo","tháng kế tiếp","tháng sau","tháng <tháng+1>","vào tháng tới"]
seasonList = "2020 đến"


In [9]:
dictList = {
    "<mean>": meanDesc,
    "<min>": minDesc,
    "<max>": maxDesc,
    "<đạt>": datList,
    "<không đạt>": koDatList,
    "<tất cả>": allList,
    "<chi tiết>": detailList,
    "<tổng quan>":overallList,
    "<tháng tiếp theo>":nextMonthList,
    "<mùa>":seasonList
}

In [39]:
tag_L = 0

ner = {
    
}

for l,g in zip(lowerDigit,lenDigit):
    tag = templateGen[l:l+g]
    
    begin = l+tag_L
    if tag == "<tháng>": end = len(month)
    elif tag == "<quý>": end = len(quarter)
    elif tag == "<năm>": end = len(year)
    elif tag == "<tên chỉ tiêu>": end = len(kpi)
    elif tag == "<tổng công ty>" or tag == "<tên tổng công ty>": end = len(comp)
    elif tag == "<tên cụm chỉ tiêu>" or tag == "<cụm chỉ tiêu>": end = len(comp)
    elif tag == "<tháng+1>": end = len(nextMonth)
    elif tag == "<năm-m>" : end = len(season)
    elif tag == "<đơn vị>": end = len(unit)
    
    
    tag_L = tag_L + end - len(tag)
    ner[tag] = (begin,end)    

In [ ]:
from collections import defaultdict

def labelNer(templateGen,data):
    def find(s, ch):
        return [i for i, ltr in enumerate(s) if ltr == ch]
    #handle tag
    lowerDigit = find(templateGen,'<')
    greaterDigit = find(templateGen,'>')
    lenDigit = [g + 1 - l for l,g in zip(lowerDigit,greaterDigit)]
    
    tag_L = 0
    ner = {}
    save = ""
    list_replace = []
    
    for l,g in zip(lowerDigit,lenDigit):
        tag = templateGen[l:l+g]
    
        begin = l+tag_L
        
        save = data.get(tag,"")
        end = len(save)
        
        tag_L = tag_L + end - len(tag)
        ner[tag] = (begin,begin+end,save)   
        list_replace.append((tag,save))
    
    #replace process
    for ele in list_replace():
        tag,save = ele
        templateGen = templateGen.replace(tag,save)
        
    #find text in templateGen
    for key in dictList.keys():
        list_ele = dictList[key]
        for ele in list_ele:
            idx = templateGen.rfind(ele)
            if idx != -1:
                ner[key] = (idx,idx+len(ele),ele)
                break
    
    return ner

In [41]:
ner

{'<tháng>': (1, 1),
 '<năm>': (3, 4),
 '<tên chỉ tiêu>': (51, 19),
 '<tổng công ty>': (76, 3)}

In [31]:
templateNew =    templateGen.replace('<tháng>',month)\
                            .replace('<năm>',year)\
                            .replace('<tên chỉ tiêu>',kpi)\
                            .replace('<tổng công ty>',comp)

In [32]:
templateNew

't1/2022 có thể cho tôi biết tường minh về chỉ tiêu tỉ lệ phản ánh CSKH được VTS điều hành nhé ?'

In [36]:
ner_evaluate = {}
ner_evaluate['<tháng>'] = (templateNew.find(month),len(month))
ner_evaluate['<năm>'] = (templateNew.find(year),len(year))
ner_evaluate['<tên chỉ tiêu>'] = (templateNew.find(kpi),len(kpi))
ner_evaluate['<tổng công ty>'] = (templateNew.find(comp),len(comp))

In [42]:
ner

{'<tháng>': (1, 1),
 '<năm>': (3, 4),
 '<tên chỉ tiêu>': (51, 19),
 '<tổng công ty>': (76, 3)}

In [38]:
ner_evaluate

{'<tháng>': (1, 1),
 '<năm>': (3, 4),
 '<tên chỉ tiêu>': (51, 19),
 '<tổng công ty>': (76, 3)}